## Loading the environments

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

# Importing packages

In [2]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from IPython.display import Image, display
from langgraph.graph import StateGraph , START , END , MessageGraph 
from langchain_core.messages import HumanMessage , AIMessage , SystemMessage
from typing import TypedDict


In [3]:
model = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

In [4]:
model.invoke("Hello, how are you?")

AIMessage(content="Hello! I'm doing well, thank you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 13, 'total_tokens': 29, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-ByqcHRj2hMVHS6CWScOM8hgKK5wXQ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--09837e53-4871-4d92-b4cc-c58b616a52a5-0', usage_metadata={'input_tokens': 13, 'output_tokens': 16, 'total_tokens': 29, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Some utility functions

In [13]:

# to display the graph
def display_graph(app):
    # this function will display the graph of the app
    try:
        display(Image(app.get_graph().draw_mermaid_png(max_retries=5, retry_delay=2.0)))
    except Exception as e:
        # This requires some extra dependencies and is optional
        print(e)



# To stream the output of the app
def stream_output(app, input):
    for output in app.stream(input):
        for key,value in output.items():
            print(f"here is output from {key}")
            print("_______")
            print(value)
            print("\n")

In [6]:
# 1. Define the state
class State(TypedDict):
    # This is a dictionary that will store the state of the workflow
    message: str

## Level 1: Simple Workflow in a graph

In [7]:
def greet_user(state):
    '''
    This function will greet the user with a message
    '''
    # Use the 'message' from the state dictionary
    user_message = state.get("message", "")
    return {"message": f"Hello {user_message}!"}


def convert_to_uppercase(state):
    '''
    This function will convert the input word in the state to uppercase
    '''
    state["message"] = state["message"].upper()
    return state

In [8]:
#define the workflow

workflow = StateGraph(State)
workflow.add_node("User_greetings" , greet_user)
workflow.add_node("Uppercase_converter" , convert_to_uppercase)
workflow.add_edge(START , "User_greetings")
workflow.add_edge("User_greetings" , "Uppercase_converter")
workflow.add_edge("Uppercase_converter" , END)
app = workflow.compile()




In [9]:
result = app.invoke({"message": "Suman"})

In [10]:
print(result)

{'message': 'HELLO SUMAN!'}


In [11]:
stream_output(app, {"message": "Suman"})

here is output from User_greetings
_______
{'message': 'Hello Suman!'}


here is output from Uppercase_converter
_______
{'message': 'HELLO SUMAN!'}




In [ ]:
#display_graph(app)

## Level 2 : Using an LLM and a function

In [15]:
def get_response_from_llm(state):
    """
    This function will get the response from the LLM
    
    """

    state["message"]  = state.get("message", "")

    response = model.invoke(state["message"]).content

    return {"message": response}


def convert_to_uppercase(state):
    """
    This function will convert the message to uppercase
    """
    response_from_llm = state["message"]
    state["message"] = response_from_llm.upper()
    
    return state



In [16]:
# define the workflow

workflow = StateGraph(State)
workflow.add_node("LLM_response" , get_response_from_llm)
workflow.add_node("Uppercase_converter" , convert_to_uppercase)
workflow.add_edge(START , "LLM_response")
workflow.add_edge("LLM_response" , "Uppercase_converter")
workflow.add_edge("Uppercase_converter" , END)
app = workflow.compile()


In [17]:
app.invoke({"message": "Hello, how are you?"})

{'message': "HELLO! I'M DOING WELL, THANK YOU. HOW CAN I ASSIST YOU TODAY?"}

In [18]:
stream_output(app, {"message": "Hello, how are you?"})

here is output from LLM_response
_______
{'message': "Hello! I'm doing well, thank you. How can I assist you today?"}


here is output from Uppercase_converter
_______
{'message': "HELLO! I'M DOING WELL, THANK YOU. HOW CAN I ASSIST YOU TODAY?"}




In [19]:
#display_graph(app)

## Level 3: real world use case

In [30]:
def classify_sentiment(state):
    """
    This function will classify the sentiment of the message
    """
    # Ensure state["message"] is a list to store message history
    if not isinstance(state.get("message", None), list):
        state["message"] = [state.get("message", "")]
    message = f"Classify the sentiment of the following message into either positive, negative or neutral: {state['message'][-1]}"
    response = model.invoke(message).content
    state["message"].append(response)
    return {"message": state["message"]}


def get_total_word_count(state):
    """
    This function will get the total word count of the message
    """
    # Get the last message (assumed to be the sentiment classification)
    last_message = state["message"][-1]
    word_count = len(last_message.split())
    state["message"].append(f"Total word count: {word_count}")
    return {"message": state["message"]}

In [31]:
workflow = StateGraph(State)
workflow.add_node("Sentiment_classifier" , classify_sentiment)
workflow.add_node("Total_word_counter" , get_total_word_count)
workflow.add_edge(START , "Sentiment_classifier")
workflow.add_edge("Sentiment_classifier" , "Total_word_counter")
workflow.add_edge("Total_word_counter" , END)
app = workflow.compile()

In [32]:
app.invoke({"message": "I am happy with the quality of the product and the service"})

{'message': ['I am happy with the quality of the product and the service',
  'The sentiment of the message is positive.',
  'Total word count: 7']}